In [1]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import warnings

# To suppress some warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import plotly.io as pio
pio.renderers.default = 'iframe'

# Corrected file path generation
filepath = [os.path.join(dirname, filename) for dirname, _, filenames in os.walk(r'C:\Users\hp\Desktop\Machine Learning\Projects\6. Coffee Shop Sales') for filename in filenames if filename.endswith('.xlsx')][0]

print(filepath)

C:\Users\hp\Desktop\Machine Learning\Projects\6. Coffee Shop Sales\Coffee Shop Sales.xlsx


In [2]:
# Read the Excel file
df = pd.read_excel(filepath, sheet_name="Transactions")

# Display the first 5 rows of the dataframe
print(df.head(5))

   transaction_id transaction_date transaction_time  transaction_qty  \
0               1       2023-01-01         07:06:11                2   
1               2       2023-01-01         07:08:56                2   
2               3       2023-01-01         07:14:04                2   
3               4       2023-01-01         07:20:24                1   
4               5       2023-01-01         07:22:41                2   

   store_id   store_location  product_id  unit_price    product_category  \
0         5  Lower Manhattan          32         3.0              Coffee   
1         5  Lower Manhattan          57         3.1                 Tea   
2         5  Lower Manhattan          59         4.5  Drinking Chocolate   
3         5  Lower Manhattan          22         2.0              Coffee   
4         5  Lower Manhattan          57         3.1                 Tea   

            product_type               product_detail  Revenue  Month Month.1  \
0  Gourmet brewed coffee     

In [3]:
# Define your color map
color_discrete_map = {"Lower Manhattan": "blue", "Hell's Kitchen": "red", "Astoria": "green"}

fig = px.scatter(df, 
                 x="transaction_date", 
                 y="Revenue",
                 color="store_location", 
                 color_discrete_map=color_discrete_map)

fig.show()

In [4]:
df[df["Revenue"] == 360]

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,Revenue,Month,Month.1,Weekday,Weekday.1,Hour
9310,9340,2023-01-17,09:05:20,8,8,Hell's Kitchen,8,45.0,Coffee beans,Premium Beans,Civet Cat,360.0,1,Jan,2,Tue,9
9365,9395,2023-01-17,09:55:47,8,8,Hell's Kitchen,8,45.0,Coffee beans,Premium Beans,Civet Cat,360.0,1,Jan,2,Tue,9
68806,68976,2023-04-17,09:55:47,8,8,Hell's Kitchen,8,45.0,Coffee beans,Premium Beans,Civet Cat,360.0,4,Apr,1,Mon,9
68981,69151,2023-04-17,11:18:31,8,8,Hell's Kitchen,8,45.0,Coffee beans,Premium Beans,Civet Cat,360.0,4,Apr,1,Mon,11
97979,98233,2023-05-17,09:05:20,8,8,Hell's Kitchen,8,45.0,Coffee beans,Premium Beans,Civet Cat,360.0,5,May,3,Wed,9
98275,98529,2023-05-17,11:18:31,8,8,Hell's Kitchen,8,45.0,Coffee beans,Premium Beans,Civet Cat,360.0,5,May,3,Wed,11
133186,133523,2023-06-17,09:55:47,8,8,Hell's Kitchen,8,45.0,Coffee beans,Premium Beans,Civet Cat,360.0,6,Jun,6,Sat,9
133337,133674,2023-06-17,10:41:11,8,8,Hell's Kitchen,8,45.0,Coffee beans,Premium Beans,Civet Cat,360.0,6,Jun,6,Sat,10
133407,133744,2023-06-17,11:18:31,8,8,Hell's Kitchen,8,45.0,Coffee beans,Premium Beans,Civet Cat,360.0,6,Jun,6,Sat,11
148702,149043,2023-06-30,11:18:31,8,8,Hell's Kitchen,8,45.0,Coffee beans,Premium Beans,Civet Cat,360.0,6,Jun,5,Fri,11


In [5]:
df_grouped = df.groupby(["transaction_date", "store_location"])[["transaction_qty", "Revenue"]].sum().reset_index()

fig = px.scatter(df_grouped, 
                 x="transaction_date", 
                 y="Revenue",
                 color="store_location",
                 color_discrete_map=color_discrete_map,
                 size="transaction_qty", 
                 trendline="ols")

fig.show()

In [6]:
df_grouped = df.groupby(["store_location", "product_category"])["Revenue"].sum().reset_index().sort_values("Revenue", ascending=False)
fig = px.bar(df_grouped, 
             x="product_category", 
             y="Revenue", 
             color="store_location",
             color_discrete_map=color_discrete_map,
             title="Product Category x Revenue")
fig.show()

In [7]:
# Function to get colors based on labels
def get_colors(labels, color_map):
    return [color_map[label] for label in labels]

fig = make_subplots(rows=1, cols=2, 
                    specs=[[{'type':'domain'}, {'type':'domain'}]], 
                    subplot_titles=["Amount of Product by Store", "Amount of Revenue by Store"])

df_grouped = df[["store_location", "product_id"]].groupby("store_location").count().reset_index()
df_grouped.columns = ['store_location', 'countage']
fig.add_trace(go.Pie(labels=df_grouped.store_location.values, 
                     values=df_grouped.countage.values, 
                     name="Product x Store",
                     marker=dict(colors=get_colors(df_grouped.store_location.values, color_discrete_map))),
              1, 1)

df_grouped = df[["store_location", "Revenue"]].groupby("store_location").sum().reset_index()
df_grouped.columns = ['store_location', 'Revenue']
fig.add_trace(go.Pie(labels=df_grouped.store_location.values, 
                     values=df_grouped.Revenue.values, 
                     name="Revenue x Store",
                     marker=dict(colors=get_colors(df_grouped.store_location.values, color_discrete_map))),
              1, 2)

fig.update_traces(hole=.2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(title_text="Global Emissions 1990-2011", width=800, height=600, showlegend=False)
fig.show()

In [8]:
df_grouped = df[['store_location', 'product_type', 'transaction_qty']].groupby(['store_location', 'product_type']).count().reset_index()
df_grouped.columns = ['store_location', 'product_type', 'countage']

fig = px.sunburst(df_grouped, 
                  path=['store_location', 'product_type'], 
                  values='countage',
                  color='store_location',
                  color_discrete_map=color_discrete_map)
fig.show()

In [9]:
# Group by 'store_location' and 'product_category' and count occurrences
df_grouped = df[['store_location', 'product_category', 'transaction_qty']].groupby(['store_location', 'product_category']).count().reset_index()
df_grouped.columns = ['store_location', 'product_category', 'countage']

# Define color map for store locations
color_discrete_map = {
    'Location1': 'blue',  # Replace with actual store locations and colors
    'Location2': 'green',
    'Location3': 'red'
    # Add more locations and colors as needed
}

# Create the sunburst chart
fig = px.sunburst(df_grouped, path=['store_location', 'product_category'], values='countage', color='store_location', color_discrete_map=color_discrete_map)

# Show the plot
fig.show()